## Required Installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install llama-index datasets accelerate bitsandbytes
!pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-huggingface

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install faiss-cpu
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 95.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


## Use your huggingface token

In [ ]:
hf_token = ""

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

## Load Mistral-7B-Instruct-v0.2 model with nf4/bitsandbytes quuantization

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

Settings.llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    # model_name = "mistralai/Mistral-7B-Instruct-v0.2",
    # tokenizer_name = "mistralai/Mistral-7B-Instruct-v0.2",
    # model_name="google/gemma-3-1b-it"
    # tokenizer_name="google/gemma-3-1b-it",
    query_wrapper_prompt=PromptTemplate("<s>[INST] {query_str} [/INST]"),
    context_window=3900,
    model_kwargs={"token": hf_token, "quantization_config": quantization_config},
    tokenizer_kwargs={"token": hf_token},
    device_map="auto",
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Settings.llm

## Use HuggingFaceEmbedding for embedding of the document chunks

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
Settings.embed_model

## Changes in the RAFTDatasetPack:

* Used Quantized mistral model instead of GPT4
* Used HuggingFaceEmbedding instead of openai embedding
* Modified ChatMessage to fit for mistral model instead of GPT4


In [ ]:
"""RAFT Dataset LlamaPack class."""

# Inspired from https://github.com/ShishirPatil/gorilla/tree/main/raft

from typing import Any, List
import random
import logging
from datasets import Dataset

# Configure logging to output to the console, with messages of level DEBUG and above
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

from llama_index.core.llama_pack.base import BaseLlamaPack
from llama_index.core import SimpleDirectoryReader

from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.embeddings.huggingface import HuggingFaceEmbedding


DEFAULT_CHUNK_SIZE = 512
DEFAULT_BREAKPOINT_PERCENTILE_THRESHOLD = 95


class RAFTDatasetPack(BaseLlamaPack):
    """RAFT Dataset Generator pack."""

    def __init__(
        self,
        file_path: str,
        llm: Any = None,
        embed_model: Any = None,
        num_questions_per_chunk: int = 5,
        num_distract_docs: int = 3,
        chunk_size: int = DEFAULT_CHUNK_SIZE,
        default_breakpoint_percentile_threshold=DEFAULT_BREAKPOINT_PERCENTILE_THRESHOLD,
    ):
        self.file_path = file_path
        self.num_questions_per_chunk = num_questions_per_chunk
        self.num_distract_docs = num_distract_docs
        self.chunk_size = chunk_size
        self.default_breakpoint_percentile_threshold = (
            default_breakpoint_percentile_threshold
        )
        self.ds = None
        self.llm = OpenAI(temperature=0, n=1, model="gpt-4") if llm is None else llm
        self.embed_model = OpenAIEmbedding() if embed_model is None else embed_model

        # self.llm = Settings.llm
        # self.embed_model = Settings.embed_model

    def strip_str(self, s) -> str:
        """
        Helper function for helping format strings returned by GPT-4.
        """
        if s.startswith("assistant:"):  # Check if the string starts with 'assistant '
            s = s.replace("assistant:", "", 1)  # Replace the first occurrence

        start_index, end_index = 0, len(s) - 1
        beg_found = False
        for i in range(len(s)):
            if s[i].isalpha():
                if not beg_found:
                    start_index = i
                    beg_found = True
                else:
                    end_index = i
        end_index += 2
        return s[start_index : min(end_index, len(s))]

    def encode_question_gen(self, question, chunk) -> List[str]:
        """
        Encode multiple prompt instructions into a single string for the general case.
        """
        prompt = f"""
            Question: {question}\nContext: {chunk}\n
            Answer this question using the information given in the context above. Here is things to pay attention to:
            - First provide step-by-step reasoning on how to answer the question.
            - In the reasoning, if you need to copy paste some sentences from the context, include them in ##begin_quote## and ##end_quote##. This would mean that things outside of ##begin_quote## and ##end_quote## are not directly copy paste from the context.
            - End your response with final answer in the form <ANSWER>: $answer, the answer should be succinct.
            You MUST begin your final answer with the tag "<ANSWER>:".
        """
        return [
            ChatMessage(
                role="user",
                content="You are a helpful question answerer who can provide an answer given a question and relevant context.",
            ),
            ChatMessage(role="assistant", content=prompt),
        ]

    def generate_label(self, question, context) -> str:
        """
        Generates the label / answer to `question` using `context` and GPT-4.
        """
        question_messages = self.encode_question_gen(question, context)
        response = self.llm.chat(question_messages)
        return str(response)

    def generate_instructions_gen(self, chunk, x=5) -> List[str]:
        """
        Generates `x` questions / use cases for `chunk`. Used when the input document is of general types
        `pdf`, `json`, or `txt`.
        """
        messages = [
            ChatMessage(
                role="user",
                content="You are a synthetic question-answer pair generator. Given a chunk of context about some topic(s), generate %s example questions a user could ask and would be answered using information from the chunk. For example, if the given context was a Wikipedia paragraph about the United States, an example question could be 'How many states are in the United States?'. The questions should be able to be answered in a few words or less."
                % (x),
            ),
            ChatMessage(role="assistant", content=str(chunk)),
        ]

        queries = str(self.llm.chat(messages)).split("\n")
        queries = [self.strip_str(q) for q in queries]
        return [q for q in queries if any(c.isalpha() for c in q)]

    def get_chunks(self, file_path: str, chunk_size: int) -> List[str]:
        """
        Takes in a `file_path`, retrieves the document, breaks it down into chunks of size
        `chunk_size`, and returns the chunks.
        """
        documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
        splitter = SemanticSplitterNodeParser(
            buffer_size=1,
            breakpoint_percentile_threshold=self.default_breakpoint_percentile_threshold,
            embed_model=self.embed_model,
        )
        nodes = splitter.get_nodes_from_documents(documents)

        return [node.get_content() for node in nodes]

    def add_chunk_to_dataset(
        self,
        chunks: List,
        chunk: str,
        x: int = 5,
        num_distract: int = 3,
        p: float = 1.0,
    ):
        """
        Given a chunk, create {Q, A, D} triplets and add them to the dataset.
        """
        i = chunks.index(chunk)
        print(chunk)
        qs = self.generate_instructions_gen(chunk, x)
        for q in qs:
            datapt = {
                "id": None,
                "type": None,
                "question": None,
                "context": None,
                "oracle_context": None,
                "cot_answer": None,
            }

            datapt["id"] = f"seed_task_{0 if not self.ds else self.ds.num_rows}"
            datapt["type"] = "general"
            datapt["question"] = q

            # add distractor docs
            docs = [chunk]
            indices = list(range(len(chunks)))
            indices.remove(i)
            for j in random.sample(indices, num_distract):
                docs.append(chunks[j])
            # decides whether to add oracle document
            oracle = random.uniform(0, 1) < p
            if not oracle:
                docs[0] = chunks[random.sample(indices, 1)[0]]
            random.shuffle(docs)

            d = {"title": [], "sentences": []}

            d["title"].append(["placeholder_title"] * (num_distract + 1))
            d["sentences"].append(docs)
            datapt["context"] = d
            datapt["oracle_context"] = chunk

            # add answer to q
            datapt["cot_answer"] = self.generate_label(q, chunk)

            # construct model instruction
            context = ""
            for doc in docs:
                context += "<DOCUMENT>" + str(doc) + "</DOCUMENT>\n"
            context += q
            datapt["instruction"] = context

            # add to dataset
            if not self.ds:
                # init ds
                datapt["id"] = [datapt["id"]]
                datapt["type"] = [datapt["type"]]
                datapt["question"] = [datapt["question"]]
                datapt["context"] = [datapt["context"]]
                datapt["oracle_context"] = [datapt["oracle_context"]]
                datapt["cot_answer"] = [datapt["cot_answer"]]
                datapt["instruction"] = [datapt["instruction"]]
                self.ds = Dataset.from_dict(datapt)
            else:
                self.ds = self.ds.add_item(datapt)

    def run(self) -> Any:
        """Run the pipeline."""
        chunks = self.get_chunks(self.file_path, self.chunk_size)

        logger.info(f"Number of chunks created: {len(chunks)}")

        self.num_distract_docs = (
            min(self.num_distract_docs, len(chunks)) - 1
        )  # should be less than number of chunks/ nodes created

        for index, chunk in enumerate(chunks):
            logger.info(f"Processing chunk: {index}")
            self.add_chunk_to_dataset(
                chunks, chunk, self.num_questions_per_chunk, self.num_distract_docs
            )

        return self.ds


## Download Sample Data

In [ ]:
# !wget --user-agent "Mozilla" "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt" -O './paul_graham_essay.txt'


## Prepare RAFT dataset using modified RAFTDatasetPack

In [ ]:
raft_dataset = RAFTDatasetPack("./paul_graham_essay.txt")
dataset = raft_dataset.run()


INFO:__main__:Number of chunks created: 4
INFO:__main__:Processing chunk: 0




What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.




INFO:__main__:Processing chunk: 1


The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in the card reader and press a button to load the program into memory and run it. The result would ordinarily be to print something on the spectacularly loud printer.

I was puzzled by the 1401. I couldn't figure out what to do with it. And in retrospect there's not much I could have done with it. The only form of input to programs was data stored on punched cards, and I didn't have any data stored on punched cards. The only other option was to do things that didn't rely on any input, like calculate approximations of pi, but I didn't know enough math to do anything interesting of that type. So I'm not surprised I can't remember any programs I wrote, because they can't have done much. My clearest memory is of the moment I learned it was possible for programs not to terminate, when one of mine didn't. On a machine without time-sharing, this was a social as well as a technical er

INFO:__main__:Processing chunk: 2


I wrote simple games, a program to predict how high my model rockets would fly, and a word processor that my father used to write at least one book. There was only room in memory for about 2 pages of text, so he'd write 2 pages at a time and then print them out, but it was a lot better than a typewriter.

Though I liked programming, I didn't plan to study it in college. In college I was going to study philosophy, which sounded much more powerful. It seemed, to my naive high school self, to be the study of the ultimate truths, compared to which the things studied in other fields would be mere domain knowledge. What I discovered when I got to college was that the other fields took up so much of the space of ideas that there wasn't much left for these supposed ultimate truths. All that seemed left for philosophy were edge cases that people in other fields felt could safely be ignored.

I couldn't have put this into words when I was 18. All I knew at the time was that I kept taking philoso

INFO:__main__:Processing chunk: 3


It wasn't happening in a class, like it was supposed to, but that was ok. For the next couple years I was on a roll. I knew what I was going to do.

For my undergraduate thesis, I reverse-engineered SHRDLU. My God did I love working on that program. It was a pleasing bit of code, but what made it even more exciting was my belief — hard to imagine now, but not unique in 1985 — that it was already climbing the lower slopes of intelligence.

I had gotten into a program at Cornell that didn't make you choose a major. You could take whatever classes you liked, and choose whatever you liked to put on your degree. I of course chose "Artificial Intelligence." When I got the actual physical diploma, I was dismayed to find that the quotes had been included, which made them read as scare-quotes. At the time this bothered me, but now it seems amusingly accurate, for reasons I was about to discover.

I applied to 3 grad schools: MIT and Yale, which were renowned for AI at the time, and Harvard, whi

In [ ]:
dataset

Dataset({
    features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction'],
    num_rows: 30
})

## Save RAFT Dataset

In [ ]:
output_path = "/content/drive/MyDrive/Colab Notebooks/RAFT/test"
# Save as .arrow format
dataset.save_to_disk(output_path)

# Save as .jsonl format
dataset.to_json(output_path + ".jsonl")


Saving the dataset (0/1 shards):   0%|          | 0/30 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

361029

## Load and view RAFT data

In [ ]:
from datasets import Dataset

ds = Dataset.from_file({output_path} + "/data-00000-of-00001.arrow")


In [ ]:
ds

Dataset({
    features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction'],
    num_rows: 30
})

In [ ]:
import pandas as pd
df=pd.DataFrame(ds)


In [ ]:
df

,id,type,question,context,oracle_context,cot_answer,instruction
0,seed_task_0,general,Questions and Answers,{'sentences': [[' What I Worked On February ...,\n\nWhat I Worked On\n\nFebruary 2021\n\nBefor...,assistant: 01. What did you work on before col...,<DOCUMENT>\n\nWhat I Worked On\n\nFebruary 202...
1,seed_task_1,general,Q: How many short stories did the author write...,{'sentences': [['The language we used was an e...,\n\nWhat I Worked On\n\nFebruary 2021\n\nBefor...,assistant: \n #1. What was the auth...,<DOCUMENT>The language we used was an early ve...
2,seed_task_2,general,A: The author wrote several short stories befo...,"{'sentences': [['I wrote simple games, a progr...",\n\nWhat I Worked On\n\nFebruary 2021\n\nBefor...,assistant: 0 answers\n\nPlease provide the que...,"<DOCUMENT>I wrote simple games, a program to p..."
3,seed_task_3,general,Q: What was the author's main focus outside of...,"{'sentences': [['I wrote simple games, a progr...",\n\nWhat I Worked On\n\nFebruary 2021\n\nBefor...,assistant: \nStep 1: Identify the question bei...,"<DOCUMENT>I wrote simple games, a program to p..."
4,seed_task_4,general,A: The author's main focus outside of school w...,{'sentences': [['The language we used was an e...,\n\nWhat I Worked On\n\nFebruary 2021\n\nBefor...,assistant: 0 answers so far.\n\n\nInstructions...,<DOCUMENT>The language we used was an early ve...
5,seed_task_5,general,Q: What was the name of the machine used by th...,{'sentences': [['It wasn't happening in a clas...,\n\nWhat I Worked On\n\nFebruary 2021\n\nBefor...,assistant: 2023-02-13 16:30:37 UTC,"<DOCUMENT>It wasn't happening in a class, like..."
6,seed_task_6,general,A: The machine used by the school district was...,{'sentences': [[' What I Worked On February ...,\n\nWhat I Worked On\n\nFebruary 2021\n\nBefor...,assistant: 01. What was the machine used by th...,<DOCUMENT>\n\nWhat I Worked On\n\nFebruary 202...
7,seed_task_7,general,Q: Where was the machine located?,{'sentences': [['The language we used was an e...,\n\nWhat I Worked On\n\nFebruary 2021\n\nBefor...,assistant: \nAnswer:\nThe machine was located ...,<DOCUMENT>The language we used was an early ve...
8,seed_task_8,general,A: The machine was located in the basement of ...,{'sentences': [[' What I Worked On February ...,\n\nWhat I Worked On\n\nFebruary 2021\n\nBefor...,assistant: 01. How did the author work on writ...,<DOCUMENT>\n\nWhat I Worked On\n\nFebruary 202...
9,seed_task_9,general,Q: What did the author and his friend do with ...,{'sentences': [['The language we used was an e...,\n\nWhat I Worked On\n\nFebruary 2021\n\nBefor...,assistant: \n\nStep-by-Step Reasoning:\n\n1. T...,<DOCUMENT>The language we used was an early ve...


In [ ]:
df.iloc[0]

,0
id,seed_task_0
type,general
question,Questions and Answers
context,{'sentences': [[' What I Worked On February ...
oracle_context,\n\nWhat I Worked On\n\nFebruary 2021\n\nBefor...
cot_answer,assistant: 01. What did you work on before col...
instruction,<DOCUMENT>\n\nWhat I Worked On\n\nFebruary 202...


In [ ]:
df.iloc[0]["question"]

'Questions and Answers'

In [ ]:
df.iloc[0]["context"]

{'sentences': [['\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\n',
   "The language we used was an early version of Fortran. You had to type programs

In [ ]:
df.iloc[0]["oracle_context"]

'\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\n'

In [ ]:
df.iloc[0]["cot_answer"]

"assistant: 01. What did you work on before college?\n\nAnswer: $BeforeCollege: Writing and programming.\n\nExplanation:\n\nFrom the context, we know that before college, the speaker worked on writing short stories and programming on the IBM 1401 at their junior high school. The speaker mentions that they wrote awful short stories with hardly any plot and imagined deep characters, indicating that their writing skills were still developing. They also mention that they and their friend Rich Draves were able to use the school district's 1401 in the basement of their junior high school, where they tried writing programs. Therefore, the answer to the question is $BeforeCollege: Writing and programming."

In [ ]:
df.iloc[0]["instruction"]

'<DOCUMENT>\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\n</DOCUMENT>\n<DOCUMENT>The language we used was an early version of Fortran. You had to typ

## Reference:
1. https://github.com/run-llama/llama_index/blob/main/llama-index-packs/llama-index-packs-raft-dataset/examples/raft_dataset.ipynb

In [ ]:
from datasets import load_dataset, Dataset
from typing import Any, Dict, List, Optional, Tuple
import hashlib

def _to_text(x: Any) -> str:
    """
    PubMedQAなどで context が str / list / dict で揺れる時に
    いい感じに 1 本のテキストへ正規化。
    """
    if x is None:
        return ""
    if isinstance(x, str):
        return x
    if isinstance(x, list):
        return " ".join([_to_text(t) for t in x if t is not None])
    if isinstance(x, dict):
        # よくある key 候補を順に試す
        for k in ["context", "contexts", "text", "abstract", "sentences"]:
            if k in x:
                return _to_text(x[k])
        # 何も無ければ全部連結
        return " ".join([_to_text(v) for v in x.values()])
    return str(x)


def chunk_text(
    text: str,
    max_tokens: Optional[int] = None,
    tokenizer=None,
) -> List[str]:
    """
    長文だけ、ゆるくチャンク分割したい場合に使う。
    max_tokens=None なら分割しない。
    tokenizer があれば トークン数ベース、なければ空白分割で近似。
    """
    text = text.strip()
    if not text:
        return []
    if max_tokens is None:
        return [text]

    if tokenizer is not None:
        ids = tokenizer.encode(text, add_special_tokens=False)
        chunks = []
        for i in range(0, len(ids), max_tokens):
            sub = ids[i:i+max_tokens]
            chunks.append(tokenizer.decode(sub, skip_special_tokens=True).strip())
        return [c for c in chunks if c]
    else:
        words = text.split()
        chunks = []
        for i in range(0, len(words), max_tokens):
            sub = words[i:i+max_tokens]
            chunks.append(" ".join(sub).strip())
        return [c for c in chunks if c]


## HotpotQA

In [ ]:
from datasets import load_dataset, Dataset
import random

def hotpot_to_raft_base_general(
    split: str = "train",
    num_distract: int = 3,
    seed: int = 42,
    max_examples: int | None = None,
    max_oracle_docs: int = 2,   # 参照チャンクとして使う最大数（1 or 2 想定）
) -> Dataset:
    rng = random.Random(seed)
    raw = load_dataset("hotpot_qa", "distractor", split=split)

    data = {
        "id": [],
        "type": [],
        "question": [],
        "context": [],
        "oracle_context": [],
        "cot_answer": [],   # あとで CoT を埋める
        "instruction": [],
        "gold_answers": [],
    }

    for idx, ex in enumerate(raw):
        if max_examples is not None and len(data["id"]) >= max_examples:
            break

        print(ex)
        q = ex["question"]
        print(q)
        context = ex["context"]      # [[title, [sent1, sent2...]], ...]
        print(context)
        supporting_facts = ex["supporting_facts"]  # [[title, sent_id], ...]
        print(supporting_facts)

        support_titles = {t for t, id in zip(supporting_facts["title"], supporting_facts["sent_id"])}

        paragraphs = []
        for title, sentences in zip(context["title"],context["sentences"]):
            text = " ".join(sentences)
            is_support = title in support_titles
            paragraphs.append(
                {"title": title, "text": text, "is_support": is_support}
            )

        gold = [p for p in paragraphs if p["is_support"]]
        non_gold = [p for p in paragraphs if not p["is_support"]]

        # ★ 参照が少なくとも 1 個はあるものだけ使う
        if len(gold) < 1:
            continue
        if len(non_gold) < num_distract:
            continue

        # 参照チャンクとして使う gold を最大 max_oracle_docs 個まで採用
        gold_docs = gold[:max_oracle_docs]
        distract = rng.sample(non_gold, 5 - len(gold_docs))

        # docs = 複数 gold + distractor
        docs = gold_docs + distract
        rng.shuffle(docs)

        docs_texts = [d["text"] for d in docs]
        titles = [d["title"] for d in docs]

        context_dict = {
            "title": [titles],
            "sentences": [docs_texts],
        }

        # oracle_context は「すべての gold 段落の結合」にしておく
        oracle_context = " ".join(p["text"] for p in gold)

        # RAFT と同じ instruction 形式
        instruction = ""
        for doc_text in docs_texts:
            instruction += "<DOCUMENT>" + doc_text + "</DOCUMENT>\n"
        instruction += q

        rec_id = f"hotpot_{split}_{idx}"

        data["id"].append(rec_id)
        data["type"].append("general")
        data["question"].append(q)
        data["context"].append(context_dict)
        data["oracle_context"].append(oracle_context)
        data["cot_answer"].append("")   # 後で埋める
        data["instruction"].append(instruction)
        data["gold_answers"].append([ex["answer"]])

    return Dataset.from_dict(data)


In [ ]:
def build_hotpot_corpus(
    split: str = "train",
    config: str = "distractor",
    dedup: bool = False,
    max_tokens: Optional[int] = None,
    tokenizer=None,
) -> Dataset:
    raw = load_dataset("hotpot_qa", config, split=split)

    docs = []
    meta_title = []

    seen = set()
    for ex in raw:
        ctx = ex["context"]  # dict
        titles = ctx["title"]
        sents_list = ctx["sentences"]

        for t, sents in zip(titles, sents_list):
            para = " ".join(sents).strip()
            if not para:
                continue

            # optional chunking
            for ch in chunk_text(para, max_tokens=max_tokens, tokenizer=tokenizer):
                if dedup:
                    h = hashlib.md5(ch.encode()).hexdigest()
                    if h in seen:
                        continue
                    seen.add(h)
                docs.append(ch)
                meta_title.append(t)

    return Dataset.from_dict({
        "retrieval_text": docs,
        "title": meta_title,
        "dataset": ["hotpotqa"] * len(docs),
        "split": [split] * len(docs),
    })


## PubMedQA

In [ ]:
from datasets import load_dataset, Dataset
import random

def pubmedqa_to_raft_base_with_distractors(
    subset: str = "pqa_labeled",   # "pqa_labeled", "pqa_artificial", "pqa_unlabeled"
    split: str = "train",
    num_distract: int = 3,
    max_examples: int | None = None,
    seed: int = 42,
) -> Dataset:
    """
    PubMedQA を RAFT 用のベース Dataset に変換する（distractor 付き）。
    - 正例 abstract + ランダムに選んだ別 abstract を distractor として追加。
    - RAFT と互換なカラム:
        id, type, question, context, oracle_context, cot_answer, instruction
      ＋ メタ情報として long_answer, final_decision を保持。
    """
    rng = random.Random(seed)
    raw = load_dataset("qiaojin/PubMedQA", subset, split=split)

    # まず全 abstract をテキスト化しておく（distractor 抽選用）
    all_docs = []
    for ex in raw:
        ctx = ex["context"]
        if isinstance(ctx, dict) and "contexts" in ctx:
            sentences = ctx["contexts"]
        else:
            continue
        all_docs.append(" ".join(sentences))

    n = len(raw)

    data = {
        "id": [],
        "type": [],
        "question": [],
        "context": [],
        "oracle_context": [],
        "cot_answer": [],      # 後で CoT を埋める
        "instruction": [],
        "long_answer": [],
        "gold_answers": [],
    }

    for idx, ex in enumerate(raw):
        if max_examples is not None and len(data["id"]) >= max_examples:
            break

        q = ex["question"]

        # 正例 abstract
        oracle_doc = all_docs[idx]

        # distractor 用に別 idx をサンプリング（自分自身は除外）
        if n - 1 < num_distract:
            # データ数が少なすぎる場合はスキップ or 無理やりに数を減らすなど
            continue

        candidate_indices = list(range(n))
        candidate_indices.remove(idx)
        distract_indices = rng.sample(candidate_indices, num_distract)
        distract_docs = [all_docs[j] for j in distract_indices]

        # docs = 正例 + distractor
        docs = [{"title": f"pubmed_{ex['pubid']}", "text": oracle_doc, "is_oracle": True}]
        for j, d_text in zip(distract_indices, distract_docs):
            docs.append({
                "title": f"pubmed_{raw[j]['pubid']}",
                "text": d_text,
                "is_oracle": False,
            })

        # モデルに「どれが正例か」を当てさせたいならシャッフル推奨
        rng.shuffle(docs)

        titles = [d["title"] for d in docs]
        docs_texts = [d["text"] for d in docs]

        # RAFT と同じ context 形式（2 次元）
        context_dict = {
            "title": [titles],
            "sentences": [docs_texts],
        }

        # oracle_context は「正例 abstract の全文」
        oracle_context = oracle_doc

        # instruction も RAFT と同じ
        instruction = ""
        for doc_text in docs_texts:
            instruction += f"<DOCUMENT>{doc_text}</DOCUMENT>\n"
        instruction += q

        rec_id = f"pubmedqa_{subset}_{idx}"

        data["id"].append(rec_id)
        data["type"].append("biomedical_qa")
        data["question"].append(q)
        data["context"].append(context_dict)
        data["oracle_context"].append(oracle_context)
        data["cot_answer"].append("")   # 後で LLM で埋める
        data["instruction"].append(instruction)
        data["long_answer"].append(ex.get("long_answer", ""))
        data["gold_answers"].append(ex.get("final_decision", ""))

    return Dataset.from_dict(data)


In [ ]:
def build_pubmedqa_corpus(
    split: str = "train",
    subset: str = "pqa_labeled",
    dedup: bool = False,
    max_tokens: Optional[int] = None,
    tokenizer=None,
) -> Dataset:
    raw = load_dataset("qiaojin/PubMedQA", subset, split=split)

    docs = []
    seen = set()

    for ex in raw:
        ctx_text = ex.get("context")
        if not ctx_text:
            continue

        for ch in chunk_text(ctx_text, max_tokens=max_tokens, tokenizer=tokenizer):
            if dedup:
                h = hashlib.md5(ch.encode()).hexdigest()
                if h in seen:
                    continue
                seen.add(h)
            docs.append(ch)

    return Dataset.from_dict({
        "retrieval_text": docs,
        "dataset": ["pubmedqa"] * len(docs),
        "split": [split] * len(docs),
    })


## SquaD

In [ ]:
from datasets import load_dataset, Dataset
import random
from typing import Optional, List, Dict, Any


def squad_to_raft_base_with_distractors(
    split: str = "train",          # "train" or "validation"
    num_distract: int = 3,         # gold1 + distractor3 みたいな感じ
    max_examples: Optional[int] = None,
    seed: int = 42,
) -> Dataset:
    """
    SQuAD (rajpurkar/squad) を RAFT 形式 (distractor 付き) に変換する。

    出力カラム:
      - id:  "squad_{split}_{orig_id}"
      - type: "squad"
      - question: SQuAD の question
      - context:  RAFT 形式のコンテキスト
          {"title": [[...]], "sentences": [[doc1, doc2, ...]]}
      - oracle_context: gold コンテキスト (元の context そのまま)
      - cot_answer: "" （後で LLM で埋める）
      - instruction: <DOCUMENT>doc</DOCUMENT> ... + question
      - gold_answers: SQuAD の answers["text"]
    """
    rng = random.Random(seed)

    raw = load_dataset("rajpurkar/squad", split=split)

    # distractor 抽選用に、全 context と title をプールしておく
    all_contexts = [ex["context"] for ex in raw]
    all_titles = [ex["title"] for ex in raw]
    n = len(raw)

    data: Dict[str, List[Any]] = {
        "id": [],
        "type": [],
        "question": [],
        "context": [],
        "oracle_context": [],
        "cot_answer": [],
        "instruction": [],
        "gold_answers": [],
    }

    for idx, ex in enumerate(raw):
        if max_examples is not None and len(data["id"]) >= max_examples:
            break

        q = ex["question"]
        ctx = ex["context"]
        title = ex["title"]
        orig_id = ex["id"]
        gold_answers = ex["answers"]["text"]

        # gold doc は「このサンプルの context」
        gold_doc = ctx
        gold_title = title

        # distractor を他サンプルからサンプリング
        docs: List[str]
        titles: List[str]

        if num_distract > 0:
            if n - 1 < num_distract:
                # データ数が少なすぎるときはスキップ or num_distract を減らす
                continue

            candidate_indices = list(range(n))
            candidate_indices.remove(idx)
            distract_indices = rng.sample(candidate_indices, num_distract)
            distract_docs = [all_contexts[j] for j in distract_indices]
            distract_titles = [all_titles[j] for j in distract_indices]

            docs = [gold_doc] + distract_docs
            titles = [gold_title] + distract_titles
        else:
            docs = [gold_doc]
            titles = [gold_title]

        # gold + distractor をシャッフル（どれが gold かモデルには教えない）
        combined = list(zip(titles, docs))
        rng.shuffle(combined)
        titles_shuf, docs_shuf = zip(*combined)

        # RAFT と同じ context 形式（2 次元）
        context_dict = {
            "title": [list(titles_shuf)],
            "sentences": [list(docs_shuf)],
        }

        # oracle_context は元の gold コンテキストそのまま
        oracle_context = gold_doc

        # instruction: <DOCUMENT>doc</DOCUMENT> ... + question
        instruction = ""
        for d in docs_shuf:
            instruction += f"<DOCUMENT>{d}</DOCUMENT>\n"
        instruction += q

        rec_id = f"squad_{split}_{orig_id}"

        data["id"].append(rec_id)
        data["type"].append("squad")
        data["question"].append(q)
        data["context"].append(context_dict)
        data["oracle_context"].append(oracle_context)
        data["cot_answer"].append("")          # ここは後で CoT で埋める
        data["instruction"].append(instruction)
        data["gold_answers"].append(gold_answers)

    return Dataset.from_dict(data)


In [ ]:
def build_squad_corpus(
    split: str = "train",
    dataset_id: str = "rajpurkar/squad",
    dedup: bool = False,
    max_tokens: Optional[int] = None,
    tokenizer=None,
) -> Dataset:
    raw = load_dataset(dataset_id, split=split)

    docs = []
    seen = set()

    for ex in raw:
        ctx = ex["context"].strip()
        if not ctx:
            continue

        for ch in chunk_text(ctx, max_tokens=max_tokens, tokenizer=tokenizer):
            if dedup:
                h = hashlib.md5(ch.encode()).hexdigest()
                if h in seen:
                    continue
                seen.add(h)
            docs.append(ch)

    return Dataset.from_dict({
        "retrieval_text": docs,
        "dataset": ["squad"] * len(docs),
        "split": [split] * len(docs),
    })


### PopQA
https://huggingface.co/datasets/Atipico1/popQA
一旦これを使うけど、本当は自分たちで作るべき

In [ ]:
from datasets import load_dataset, Dataset
import random
from typing import Optional, List, Dict, Any


def popqa_to_raft_base_with_distractors(
    split: str = "train",          # "train" or "test"
    num_distract: int = 4,         # gold + distractor を何個にするか
    max_oracle_docs: int = 1,      # hasanswer=True の ctx を何個まで入れるか
    max_examples: Optional[int] = None,
    seed: int = 42,
) -> Dataset:
    """
    Atipico1/popQA を RAFT 形式 (distractor 付き) の HF Dataset に変換する。

    出力カラム:
      - id:  "popqa_{split}_{orig_id}"
      - type: "popqa"
      - question: popQA の question
      - context:  RAFT 形式のコンテキスト
          {"title": [[title1, ...]], "sentences": [[text1, ...]]}
      - oracle_context: hasanswer=True の ctx.text を全部 concat
      - cot_answer: "" （後で LLM で埋める）
      - instruction: <DOCUMENT>doc</DOCUMENT> ... + question
      - gold_answers: ex["answers"] （PopQA のゴールド回答）
    """
    rng = random.Random(seed)

    raw = load_dataset("Atipico1/popQA", split=split)

    data: Dict[str, List[Any]] = {
        "id": [],
        "type": [],
        "question": [],
        "context": [],
        "oracle_context": [],
        "cot_answer": [],
        "instruction": [],
        "gold_answers": [],
    }

    for ex in raw:
        if max_examples is not None and len(data["id"]) >= max_examples:
            break

        q = ex["question"]
        orig_id = ex["id"]          # int64
        answers = ex["answers"]     # list[str]
        ctxs: List[Dict[str, Any]] = ex["ctxs"]

        if not ctxs:
            continue

        # gold docs: hasanswer=True
        gold_ctxs = [c for c in ctxs if c.get("hasanswer", False)]
        # distractor 候補: hasanswer=False
        neg_ctxs = [c for c in ctxs if not c.get("hasanswer", False)]

        # gold が 1つもないサンプルはスキップ
        if len(gold_ctxs) == 0:
            continue

        # oracle_docs = gold_ctxs のうち max_oracle_docs 個までランダムに取得する
        oracle_docs = rng.sample(gold_ctxs, k=min(max_oracle_docs, len(gold_ctxs)))

        # distractor をローカル neg_ctxs からサンプリング
        distractors: List[Dict[str, Any]] = []
        if num_distract > 0 and len(neg_ctxs) > 0:
            k = min(num_distract, len(neg_ctxs))
            distractors = rng.sample(neg_ctxs, k=k)

        # 最終的に context に入れる doc セット
        docs_for_context = oracle_docs + distractors
        # 念のため重複排除（同じ text の ctx が混ざる可能性があるため）
        seen_text = set()
        uniq_docs = []
        for d in docs_for_context:
            t = d.get("text", "")
            if not t or t in seen_text:
                continue
            seen_text.add(t)
            uniq_docs.append(d)

        if not uniq_docs:
            continue

        # シャッフル（どれが gold かはモデルには教えない）
        rng.shuffle(uniq_docs)

        titles = [d.get("title", "") for d in uniq_docs]
        texts = [d.get("text", "") for d in uniq_docs]

        # RAFT 形式の context （2次元）
        context_dict = {
            "title": [titles],
            "sentences": [texts],
        }

        # oracle_context = すべての gold_ctx.text を concat
        oracle_context = " ".join(
            d.get("text", "") for d in oracle_docs if d.get("text", "")
        ).strip()

        if not oracle_context:
            # ありえないと思うが念のため
            continue

        # instruction: <DOCUMENT>text</DOCUMENT> ... + question
        instruction = ""
        for t in texts:
            instruction += f"<DOCUMENT>{t}</DOCUMENT>\n"
        instruction += q

        rec_id = f"popqa_{split}_{orig_id}"

        data["id"].append(rec_id)
        data["type"].append("popqa")
        data["question"].append(q)
        data["context"].append(context_dict)
        data["oracle_context"].append(oracle_context)
        data["cot_answer"].append("")          # ここは後で CoT を埋める
        data["instruction"].append(instruction)
        data["gold_answers"].append(list(answers))

    return Dataset.from_dict(data)


In [ ]:
from datasets import load_dataset, Dataset
import hashlib

def build_popqa_corpus_atipico1(
    split: str = "train",
    dedup: bool = False,
):
    raw = load_dataset("Atipico1/popQA", split=split)

    docs = []
    titles = []
    hasanswer_flags = []

    seen = set()
    for ex in raw:
        ctxs = ex.get("ctxs", [])
        for c in ctxs:
            text = (c.get("text") or "").strip()
            if not text:
                continue

            if dedup:
                h = hashlib.md5(text.encode()).hexdigest()
                if h in seen:
                    continue
                seen.add(h)

            docs.append(text)
            titles.append(c.get("title", ""))
            hasanswer_flags.append(bool(c.get("hasanswer", False)))

    return Dataset.from_dict({
        "retrieval_text": docs,      # 1 doc = 1行
        "title": titles,
        "hasanswer": hasanswer_flags,
        "dataset": ["popqa"] * len(docs),
        "split": [split] * len(docs),
    })


In [ ]:
# CORPUS_BUILDERS = {
#     "hotpotqa": build_hotpot_corpus,
#     "pubmedqa": build_pubmedqa_corpus,
#     "squad": build_squad_corpus,
#     "popqa": build_popqa_corpus,
# }

# def build_doc_corpus(
#     dataset_name: str,
#     split: str = "train",
#     out_dir: Optional[str] = None,
#     **kwargs,
# ) -> Dataset:
#     name = dataset_name.lower()
#     if name not in CORPUS_BUILDERS:
#         raise ValueError(f"Unknown dataset_name={dataset_name}. choose from {list(CORPUS_BUILDERS)}")

#     corpus = CORPUS_BUILDERS[name](split=split, **kwargs)

#     if out_dir is not None:
#         os.makedirs(out_dir, exist_ok=True)
#         save_path = os.path.join(out_dir, f"{name}_{split}_corpus")
#         corpus.save_to_disk(save_path)
#         corpus.to_json(os.path.join(save_path, "corpus.jsonl"))
#         print(f"Saved corpus to: {save_path}")

#     return corpus


### CoTの作成

In [ ]:

def add_cot_with_raft_pubmed(example):
    raft_pack = RAFTDatasetPack(file_path="dummy.txt")
    cot = raft_pack.generate_label(
        example["question"],
        example["oracle_context"],   # 正例 abstract 全文
    )
    example["cot_answer"] = str(cot)
    return example

### RAG用のembedding作成

In [ ]:
DATASET_BUILDERS = {
    "hotpotqa": hotpot_to_raft_base_general,
    "pubmedqa": pubmedqa_to_raft_base_with_distractors,
    "squad":    squad_to_raft_base_with_distractors,
    "popqa":    popqa_to_raft_base_with_distractors,
}

In [ ]:
import os
from datasets import load_from_disk

def build_hotpot_raft_with_retriever_and_save(
    dataset_name: str,
    out_dir: str,
    split: str = "train",
    num_distract: int = 3,
    seed: int = 42,
    max_examples: int | None = None,
    max_oracle_docs: int = 2,
    embed_model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
):
    # 1) まず従来通りRAFTベースを作る
    if dataset_name not in DATASET_BUILDERS:
        raise ValueError(f"Unknown dataset_name={dataset_name}. choose from {list(DATASET_BUILDERS)}")

    ds = DATASET_BUILDERS[dataset_name](
        split=split,
        num_distract=num_distract,
        seed=seed,
        max_examples=max_examples,
        # max_oracle_docs=max_oracle_docs,
    )

    # cot を与える
    ds_cot = ds.map(add_cot_with_raft_pubmed)

    # 3) 保存
    os.makedirs(out_dir, exist_ok=True)
    ds_cot.save_to_disk(out_dir)
    # ついでに jsonl も残すなら
    ds_cot.to_json(os.path.join(out_dir, "dataset.jsonl"))

    print(f"Saved to: {out_dir}")
    return ds_cot


In [ ]:
ds = build_hotpot_raft_with_retriever_and_save(dataset_name="hotpotqa", out_dir="/content/drive/MyDrive/Colab Notebooks/RAFT/test", max_examples = 5 )

{'id': '5a7a06935542990198eaf050', 'question': "Which magazine was started first Arthur's Magazine or First for Women?", 'answer': "Arthur's Magazine", 'type': 'comparison', 'level': 'medium', 'supporting_facts': {'title': ["Arthur's Magazine", 'First for Women'], 'sent_id': [0, 0]}, 'context': {'title': ['Radio City (Indian radio station)', 'History of Albanian football', 'Echosmith', "Women's colleges in the Southern United States", 'First Arthur County Courthouse and Jail', "Arthur's Magazine", '2014–15 Ukrainian Hockey Championship', 'First for Women', 'Freeway Complex Fire', 'William Rast'], 'sentences': [["Radio City is India's first private FM radio station and was started on 3 July 2001.", ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).', ' It plays Hindi, English and regional songs.', ' It was launched in Hyderabad in March 2006, in Chennai on 7 Ju

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Saved to: /content/drive/MyDrive/Colab Notebooks/RAFT/test


In [ ]:
ds["cot_answer"]

Column(["assistant: Reasoning: \n\nTo answer this question, we need to compare the start dates of both magazines. \n\nFrom the context, we can see that ##begin_quote## Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. ##end_quote## This tells us that Arthur's Magazine was started in 1844.\n\nOn the other hand, the context also tells us that ##begin_quote## First for Women is a woman's magazine published by Bauer Media Group in the USA. The magazine was started in 1989. ##end_quote## This means that First for Women was started in 1989.\n\nComparing these two dates, it's clear that Arthur's Magazine was started first.\n\n<ANSWER>: Arthur's Magazine was started first.", 'assistant: To answer the question, we need to identify where the head office of The Oberoi Group is located. The context provides this information directly. \n\n##begin_quote##\nThe Oberoi Group is a hotel company with its head office in Delhi.\n##end_quote##\

In [ ]:
import os
import numpy as np
import faiss
from typing import Optional, Tuple, Dict, Any
from datasets import Dataset
from sentence_transformers import SentenceTransformer

CORPUS_BUILDERS = {
    "hotpotqa": build_hotpot_corpus,
    "pubmedqa": build_pubmedqa_corpus,
    "squad": build_squad_corpus,
    "popqa": build_popqa_corpus_atipico1,  # Atipico1版に差し替えてOK
}

def ensure_doc_index(
    corpus: Dataset,
    embed_model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
    text_column: str = "retrieval_text",
    batch_size: int = 64,
    force_recompute: bool = False,
):
    """
    corpus に embedding と faiss index を保証する。
    - embedding があり & force_recompute=False → embedding再利用で index だけ作る
    - embedding が無い or force_recompute=True → embedding作り直して index張る
    戻り値: (corpus_indexed, embed_model)
    """
    embed_model = SentenceTransformer(embed_model_name)

    # 1) embedding が既にあり、再計算しないなら → indexだけ作り直す
    if (not force_recompute) and ("embedding" in corpus.column_names):
        embeddings = np.asarray(corpus["embedding"], dtype="float32")
        faiss.normalize_L2(embeddings)
        dim = embeddings.shape[1]
        index = faiss.IndexFlatIP(dim)
        index.add(embeddings)
        corpus = corpus.add_faiss_index(column="embedding", custom_index=index)
        return corpus, embed_model

    # 2) embedding を作る
    texts = corpus[text_column]
    embeddings = embed_model.encode(texts, batch_size=batch_size, show_progress_bar=True)
    embeddings = np.asarray(embeddings, dtype="float32")
    faiss.normalize_L2(embeddings)

    corpus = corpus.add_column("embedding", embeddings.tolist())

    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    corpus = corpus.add_faiss_index(column="embedding", custom_index=index)

    return corpus, embed_model


def build_doc_corpus(
    dataset_name: str,
    split: str = "train",
    out_dir: Optional[str] = None,
    # ====== ここから embedding/index 関連の追加引数 ======
    add_embeddings: bool = True,          # CHANGE: corpus作成後にembedding/indexを付けるか
    embed_model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
    batch_size: int = 64,
    reuse_embedding: bool = False,         # CHANGE: embedding列があれば再利用する
    force_recompute: bool = True,        # CHANGE: Trueならembeddingを作り直す
    # ====================================================
    **kwargs,
) -> Dataset:
    """
    dataset_name に対応する doc corpus (1行=1doc) を作り、
    必要なら embedding + FAISS index を付与して保存する。
    """

    name = dataset_name.lower()
    if name not in CORPUS_BUILDERS:
        raise ValueError(f"Unknown dataset_name={dataset_name}. choose from {list(CORPUS_BUILDERS)}")

    # 0) dataset別 corpus を作る（1行=1doc）
    corpus = CORPUS_BUILDERS[name](split=split, **kwargs)

    # 1) embedding/index を付与
    if add_embeddings:
        # reuse_embedding=False なら「embeddingがあっても無視して作り直す」
        _force = force_recompute or (not reuse_embedding)
        corpus, _ = ensure_doc_index(
            corpus,
            embed_model_name=embed_model_name,
            text_column="retrieval_text",
            batch_size=batch_size,
            force_recompute=_force,
        )

    # 2) 保存
    if out_dir is not None:
        os.makedirs(out_dir, exist_ok=True)
        save_path = os.path.join(out_dir, f"{name}_{split}_corpus")
        corpus.save_to_disk(save_path)
        corpus.to_json(os.path.join(save_path, "corpus.jsonl"))
        print(f"Saved corpus to: {save_path}")

    return corpus


In [ ]:
corpus = build_doc_corpus(dataset_name="hotpotqa", split="validation", out_dir = "/content/drive/MyDrive/Colab Notebooks/RAFT/test")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1152 [00:00<?, ?it/s]

ImportError: You must install Faiss to use FaissIndex. To do so you can run `conda install -c pytorch faiss-cpu` or `conda install -c pytorch faiss-gpu`. A community supported package is also available on pypi: `pip install faiss-cpu` or `pip install faiss-gpu`. Note that pip may not have the latest version of FAISS, and thus, some of the latest features and bug fixes may not be available.